# Import

In [ ]:
import optuna
from optuna.visualization import plot_edf, plot_slice, plot_optimization_history, plot_param_importances

# DeterministicProcess params

In [ ]:
df_full = pd.concat([df_train, df_test], axis=0)
    
def objective(trial):
    
    fourier_order = trial.suggest_int("fourier_order", 0, 20)
    constant = trial.suggest_categorical("constant", [True, False])
    dp_order = trial.suggest_int("dp_order", 1, 3)
    seasonal = trial.suggest_categorical("seasonal", [True, False])
    
    #     Prepare datasets with new params
    df_full_prep = data_prep_optuna(df_full, fourier_order, constant, dp_order, seasonal)
    features = df_full_prep.columns
    df_full_prep['date'] = df_full.date
    df_full_prep['num_sold'] = df_full.num_sold.astype(np.float32)

    df_train_prep = df_full_prep[df_full_prep.date.dt.year < 2018]
    df_val_prep = df_full_prep[df_full_prep.date.dt.year == 2018]
    df_test_prep = df_full_prep[df_full_prep.date.dt.year == 2019]
    
    #     Eval model
    preproc, model, y_va_pred = fit_simple_model(df_train_prep, X_va=df_val_prep, outliers=False, features=features)
    y_va_true = df_val_prep.num_sold.values.reshape(-1, 1)
    score = f_smape(y_va_true, y_va_pred)

    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

df_study = study.trials_dataframe()
df_study.to_excel('study_DeterministicProcess_params.xlsx')

best_params = study.best_params
best_params

# Models optimization

## ExtraTreesRegressor

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 50,200),
#         'criterion': trial.suggest_categorical("criterion", ['squared_error', 'absolute_error']),
#         'max_depth': trial.suggest_int("max_depth", 3,15),
#         'min_samples_split': trial.suggest_int("min_samples_split", 2, 5),
#         'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 5),
        'max_features': trial.suggest_categorical("max_features", ['auto', 'sqrt', 'log2']),
#         'bootstrap': trial.suggest_categorical("bootstrap", [True, False]),
        'n_jobs': -1,
        'random_state': 0,
        }
    
    model = ExtraTreesRegressor(**params)
    score = modelisation_TT_scoring(model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

df_study = study.trials_dataframe()
df_study.to_excel('study_ET.xlsx')

ET_params = study.best_params
ET_params

## NuSVR

In [ ]:
def objective(trial):
    # default params : nu=0.5, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, tol=0.001, cache_size=200, verbose=False, max_iter=- 1)
    params = {
        'nu' : trial.suggest_float("nu", 0.2, 0.5), 
        'C' : trial.suggest_float("C", 0.1, 1),
        'kernel' : trial.suggest_categorical("kernel", ['linear', 'poly', 'rbf']),
#         'kernel':'linear',
#         'degree' : trial.suggest_int("degree", 2, 4),
#         'tol' : trial.suggest_float("tol", 5e-4, 1e-2, log=True),
        'tol':5e-4,
#         'max_iter': 30000,
    }
    
    model = NuSVR(**params)
    
    score = modelisation_TT_scoring_NuSVR (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

df_study = study.trials_dataframe()
df_study.to_excel('study_NuSVR.xlsx')

NuSVR_params = study.best_params
NuSVR_params

## XGBRegressor

In [ ]:
def objective(trial):
    
    params = {
        'learning_rate' : trial.suggest_float("learning_rate", 0.05, 1),
        'max_depth' : trial.suggest_int("max_depth", 3, 15),
        'min_child_weight' : trial.suggest_int("min_child_weight", 1, 7),
        'gamma' : trial.suggest_float("gamma", 0, 1),
        'colsample_bytree' : trial.suggest_float("colsample_bytree", 0.3, 1),
        'colsample_bylevel' : trial.suggest_float("colsample_bylevel", 0.3, 1),
        'colsample_bynode' : trial.suggest_float("colsample_bynode", 0.3, 1),
        'n_estimators' : trial.suggest_int("n_estimators", 50, 250),
        'n_jobs' : -1,
             }
    model = XGBRegressor(**params)
    score = modelisation_TT_scoring (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

df_study = study.trials_dataframe()
df_study.to_excel('study_XGB.xlsx')

XGB_params = study.best_params
XGB_params

## RandomForestRegressor

In [ ]:
def objective(trial):
#     n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
#     max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, 
#     verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None    
    params = {
        'n_estimators' : trial.suggest_int("n_estimators", 50, 200),
    #     'criterion' : trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "poisson"]),
    #     'max_depth' : trial.suggest_int("max_depth", 3, 15),
    #     'min_samples_split' : trial.suggest_int("min_samples_split", 2, 5),
    #     'min_samples_leaf' : trial.suggest_int("min_samples_leaf", 1, 5),
        'n_jobs':-1,
             }
    
    model = RandomForestRegressor(**params)
    
    score = modelisation_TT_scoring (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

df_study = study.trials_dataframe()
df_study.to_excel('study_RF.xlsx')

RF_params = study.best_params
RF_params

# MLPClassifier

In [ ]:
warnings.simplefilter(action='ignore', category=UserWarning)

N_FOLDS = 3
subsample_pr = 0.8

df_mlp = pd.DataFrame(Normalizer().fit_transform(df), columns=cols)
X = df_mlp.loc[index_trusted]
y = pd.Series(pred_method_2[index_trusted])

def objective(trial):
    # hidden_layer_1 = int(trial.suggest_categorical("hidden_layer_1", [1, 2, 4, 8, 16, 32, 64, 128]))
    # hidden_layer_2 = int(trial.suggest_categorical("hidden_layer_2", [0, 1, 2, 4, 8, 16, 32, 64, 128]))
    # hidden_layer_3 = int(trial.suggest_categorical("hidden_layer_3", [0, 1, 2, 4, 8, 16, 32, 64, 128]))
    hidden_layer_1 = 2 ** trial.suggest_int("hidden_layer_1", 0, 7)
    hidden_layer_2 = 2 ** trial.suggest_int("hidden_layer_2", 0, 7)
    hidden_layer_3 = 2 ** trial.suggest_int("hidden_layer_3", 0, 7)
    if hidden_layer_2 == 1:
        hidden_layer_sizes = (hidden_layer_1)
    elif hidden_layer_3 == 1:
        hidden_layer_sizes = (hidden_layer_1, hidden_layer_2)
    else:
        hidden_layer_sizes = (hidden_layer_1, hidden_layer_2, hidden_layer_3)
        
    print(hidden_layer_sizes)
    
    params_MLP = {    
        'hidden_layer_sizes' : hidden_layer_sizes,
        # 'activation' : trial.suggest_categorical("activation", ['identity', 'logistic', 'tanh', 'relu']),
        # 'activation' : trial.suggest_categorical("activation", ['logistic', 'tanh']),
#         'activation': 'logistic',
        # 'solver' : trial.suggest_categorical("solver", ['lbfgs', 'sgd', 'adam']), #lbfgs is the best choice for small dataset
        # 'solver': 'adam',
        # 'tol' : trial.suggest_float("tol", 5e-5, 1e-3, log=True),
        # 'alpha' : trial.suggest_float("alpha", 5e-4, 1e-2, log=True),
        # 'learning_rate' : trial.suggest_categorical("learning_rate", ['constant', 'invscaling', 'adaptive']),
#         'learning_rate': 'adaptive',
        # 'learning_rate_init': trial.suggest_float("learning_rate_init", 1e-5, 1e-2, log=True),
#         'max_iter': trial.suggest_int("max_iter", 50, 500),
        'max_iter': 1000,
#         'beta_1': trial.suggest_float("beta_1", 0.1, 1),
#         'beta_2': trial.suggest_float("beta_2", 0.1, 1),
        'early_stopping': True,
        # 'random_state': 0,
        'warm_start': True,
             }

    score = 0
    for seed in range(N_FOLDS):
        subsample_X, _, subsample_y, _ = train_test_split(X, y, train_size=subsample_pr, random_state=seed)
        model = MLPClassifier(**params_MLP)
        model.fit(subsample_X, subsample_y)
        # print(model.score(subsample_X, subsample_y), model.score(X, y), model.score(df_mlp, pred_method_2))
        score += model.score(df_mlp, pd.Series(pred_method_2))
    return score / N_FOLDS

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

df_study = study.trials_dataframe()
df_study.to_excel('study_MLP.xlsx')

MLP_params = study.best_params
MLP_params

## MLPRegressor

In [ ]:
def objective(trial):
    params = {    
        # 'hidden_layer_sizes' : trial.suggest_int("hidden_layer_sizes", 50, 300),
        'hidden_layer_sizes': 250,
        # 'activation' : trial.suggest_categorical("activation", ['identity', 'logistic', 'tanh', 'relu']),
        'activation' : trial.suggest_categorical("activation", ['logistic', 'tanh']),
#         'activation': 'logistic',
        # 'solver' : trial.suggest_categorical("solver", ['lbfgs', 'sgd', 'adam']), #lbfgs is the best choice for small dataset
        'solver': 'adam',
        'tol' : trial.suggest_float("tol", 5e-5, 1e-3, log=True),
        'alpha' : trial.suggest_float("alpha", 5e-4, 1e-2, log=True),
        'learning_rate' : trial.suggest_categorical("learning_rate", ['constant', 'invscaling', 'adaptive']),
#         'learning_rate': 'adaptive',
        'learning_rate_init': trial.suggest_float("learning_rate_init", 1e-5, 1e-2, log=True),
#         'max_iter': trial.suggest_int("max_iter", 50, 500),
        'max_iter': 1000,
#         'beta_1': trial.suggest_float("beta_1", 0.1, 1),
#         'beta_2': trial.suggest_float("beta_2", 0.1, 1),
        'early_stopping': True,
        'random_state': 0,
        'warm_start': True,
             }
    
    model = MLPRegressor(**params)
    score = modelisation_TT_scoring (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

df_study = study.trials_dataframe()
df_study.to_excel('study_MLP.xlsx')

MLP_params = study.best_params
MLP_params

## LGBMRegressor

In [ ]:
def objective(trial):
    
    params = {
        'boosting_type': trial.suggest_categorical("boosting_type", ['gbdt', 'dart', 'goss', 'rf']), # default=gbdt
        'num_leaves': trial.suggest_int("num_leaves", 7, 50), # default=31
#         'max_depth' : trial.suggest_int("max_depth", 3, 15), # default=-1
        'learning_rate' : trial.suggest_float("learning_rate", 0.01, 0.5), # default=0.1
#         'n_estimators' : trial.suggest_int("n_estimators", 50, 250), # default=100
#         'min_child_weight' : trial.suggest_float("min_child_weight", 1e-4, 1e-2, log=True), # default=1e-3
        'colsample_bytree' : trial.suggest_float("colsample_bytree", 0.3, 1), # default=1.
        'random_state': 0,
        'n_jobs' : -1,
             }
    model = LGBMRegressor(**params)
    score = modelisation_TT_scoring (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

df_study = study.trials_dataframe()
df_study.to_excel('study_LGBM.xlsx')

LGBM_params = study.best_params
LGBM_params

## MLExtend stacking regressor

In [ ]:
def evaluate_model(model, X_tr, X_va):
    
    # Preprocess the data
    X_tr_f = X_tr[features]
    # preproc = MinMaxScaler()
    preproc = StandardScaler()
    X_tr_f = preproc.fit_transform(X_tr_f)
    y_tr = X_tr.num_sold.values.reshape(-1, 1)
    
    # Train the model
    model.fit(X_tr_f, np.log(y_tr).ravel())
    y_tr_pred = np.ceil(np.exp(model.predict(X_tr_f)).reshape(-1, 1))
    score_tr = f_smape(y_tr, y_tr_pred)  
    
    y_va_pred = None

    if X_va is not None:
        # Preprocess the validation data
        X_va_f = X_va[features]
        X_va_f = preproc.transform(X_va_f)
        y_va = X_va.num_sold.values.reshape(-1, 1)

        # Inference for validation
        y_va_pred = np.ceil(np.exp(model.predict(X_va_f)).reshape(-1, 1))
        
        # Print SMAPE and plot graph
        # eval_plot (y_va, y_va_pred)
    
    #     Eval model
    y_va_true = X_va.num_sold.values.reshape(-1, 1)
    score_val = f_smape(y_va_true, y_va_pred)  
    
    return score_tr, score_val, y_tr_pred, y_va_pred

    
def objective(trial):
    models = {
    'lr' : LinearRegression(),
    'ridge' : Ridge(),
    'huber' : HuberRegressor(),
    'sgdr' : SGDRegressor(),
    'mlp' : MLPRegressor(solver = 'lbfgs', activation = 'identity'),
    'br' : BayesianRidge(),
    'lgb' : LGBMRegressor(),
    }
    
    models_selection = {    
        'lr' : trial.suggest_categorical("lr", [False, True]),
        'ridge' : trial.suggest_categorical("ridge", [False, True]),
        'huber' : trial.suggest_categorical("huber", [False, True]),
        'sgdr' : trial.suggest_categorical("sgdr", [False, True]),
        'mlp' : trial.suggest_categorical("mlp", [False, True]),
        'br' : trial.suggest_categorical("br", [False, True]),
    }
    
    meta_models = {
    'lr' : LinearRegression(),
    'ridge' : Ridge(),
    'huber' : HuberRegressor(),
    'sgdr' : SGDRegressor(),
    'mlp' : MLPRegressor(solver = 'lbfgs', activation = 'identity'),
    'br' : BayesianRidge(),
    'lgb' : LGBMRegressor(),
    }
    
    regressors = []
    for key, value in models_selection.items():
        if value:
            regressors.append(models[key])
    
    if len(regressors) == 0:
        regressors = [LinearRegression()]
            
    use_features_in_secondary = trial.suggest_categorical("use_features_in_secondary", [False, True])
    meta_regressor_selection = trial.suggest_categorical(
        "meta_regressor_selection",
        ['lr', 'ridge', 'huber', 'sgdr', 'mlp', 'br', 'lgb']
    )
                                    
    model = mlxtend.regressor.StackingRegressor(
        regressors=regressors, 
        meta_regressor=meta_models[meta_regressor_selection], 
        use_features_in_secondary=use_features_in_secondary,
    )
    score_tr, score_eval, pred_tr, y_va_pred = evaluate_model(model, df_train_prep, df_val_prep)

    score = score_tr + score_eval
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

df_study = study.trials_dataframe()
df_study.to_excel('study_MLP.xlsx')

MLP_params = study.best_params
MLP_params

## Feature selection

In [ ]:
bad_features = ['row_id','time',TARGET, 'test']
POSSIBLE_FEATURES = [feat for feat in df.columns if feat not in bad_features]

def objective(trial):
    ###################################
    # Generate our trial model.
    ###################################
    FEATURES = []
    for feat in POSSIBLE_FEATURES:
        select_feat = trial.suggest_categorical(feat, [True, False])
        if select_feat:
            FEATURES.append(feat)
    model = LGBMRegressor()
    only_test = trial.suggest_categorical('only_test_day',[True,False])
    if only_test:
        msk = train.time.dt.weekday.isin([0])
    else:
        msk = [True for i in range(train.shape[0])]
    model.fit(train.loc[msk, FEATURES], train.loc[msk, TARGET])
    
    
    #Val Score
    val_preds = model.predict(val[FEATURES])
    score = np.mean(np.abs(val[TARGET].values - val_preds))    
    return score

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# LogisticRegression

In [ ]:
import optuna
X = X_train.copy()
# X = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X)
X = PowerTransformer().fit_transform(X)
cv = RepeatedStratifiedKFold(n_repeats=20, n_splits=10, random_state=SEED)
def objective(trial):

    solver_penalty = trial.suggest_categorical("solver_penalty", ['liblinear_l1', 'liblinear_l2', 
                                                                  'saga_l1', 'saga_l2', 'saga_elasticnet', 'saga_none',
                                                                  'newton-cg_l2', 'newton-cg_none',
                                                                  'lbfgs_l2', 'lbfgs_none',
                                                                  'sag_l2', 'sag_none',
                                                                  'newton-cholesky_l2', 'newton-cholesky_none'])                                                        
    
    params = {
        'solver': solver_penalty.split('_')[0], # default=lbfgs
        'penalty': solver_penalty.split('_')[1], # default=l2
        'C': trial.suggest_float("C", 1e-4, 1e4, log=True), # default=1.0
        # 'max_iter': trial.suggest_int("max_iter", 100, 1000), # default=100
        'random_state': SEED,
        'l1_ratio': trial.suggest_float("l1_ratio", 0, 1), # default=None
        # 'n_jobs' : -1,
        }

    if params['penalty'] != 'elasticnet':
        params['l1_ratio'] = None

    model = LogisticRegression(**params)
    score = cross_val_score(model, X, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    print(f'ROC AUC: {score.mean():.4f} +/- {score.std():.4f}')
    return score.mean() - 2 * score.std()

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500)

df_study = study.trials_dataframe()
df_study.to_excel('study_LR.xlsx')

LGBM_params = study.best_params
LGBM_params

[I 2023-04-13 11:32:20,099] A new study created in memory with name: no-name-115914e4-130a-4f09-bae1-eeb13c4b1959
[I 2023-04-13 11:32:24,340] Trial 0 finished with value: 0.5974785248619706 and parameters: {'solver_penalty': 'lbfgs_l2', 'C': 0.0008001215659820334, 'l1_ratio': 0.17670579688640475}. Best is trial 0 with value: 0.5974785248619706.


ROC AUC: 0.7493 +/- 0.0759


[I 2023-04-13 11:32:24,654] Trial 1 finished with value: 0.6432341995625586 and parameters: {'solver_penalty': 'newton-cg_l2', 'C': 0.460790495771823, 'l1_ratio': 0.23597090850515656}. Best is trial 1 with value: 0.6432341995625586.


ROC AUC: 0.7855 +/- 0.0711


[I 2023-04-13 11:32:24,890] Trial 2 finished with value: 0.6440682617554017 and parameters: {'solver_penalty': 'saga_elasticnet', 'C': 1.675046069786646, 'l1_ratio': 0.7880294483420687}. Best is trial 2 with value: 0.6440682617554017.
[I 2023-04-13 11:32:25,096] Trial 3 finished with value: 0.6456207833961292 and parameters: {'solver_penalty': 'liblinear_l1', 'C': 0.4436128986522135, 'l1_ratio': 0.3926265033571048}. Best is trial 3 with value: 0.6456207833961292.


...

[I 2023-04-13 11:34:21,195] Trial 498 finished with value: 0.6511821366828551 and parameters: {'solver_penalty': 'liblinear_l1', 'C': 0.027707181737206717, 'l1_ratio': 0.130098374830769}. Best is trial 23 with value: 0.6511821366828551.


ROC AUC: 0.7902 +/- 0.0695


[I 2023-04-13 11:34:21,440] Trial 499 finished with value: 0.3878333612163146 and parameters: {'solver_penalty': 'liblinear_l1', 'C': 0.009290207971706634, 'l1_ratio': 0.13024482717012467}. Best is trial 23 with value: 0.6511821366828551.


ROC AUC: 0.6428 +/- 0.1275


{'solver_penalty': 'saga_l1',
 'C': 0.04349403065199462,
 'l1_ratio': 0.569760298788535}

# Plot

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

In [ ]:
optuna.visualization.plot_slice(study)